In [1]:
from harness import Harness

import sklearn.datasets
import sklearn.discriminant_analysis
import sklearn.mixture
import sklearn.model_selection

from pandas import ( 
    CategoricalIndex, DataFrame, Index, Series
)

from IPython.display import (
    Markdown,
)


In [2]:
"""Can I load the iris data"""
iris = sklearn.datasets.load_iris()

In [3]:
"""Can I initialize a Harness DataFrame"""
df = Harness(
    data=iris['data'], 
    index=CategoricalIndex(iris['target']).rename('target'),
    columns=iris['feature_names'],
    estimator=sklearn.discriminant_analysis.LinearDiscriminantAnalysis(),
    feature_level='target',
)

df = df.set_index(
    df.index
    .rename_categories(iris['target_names'])
    .rename('target_name'), append=True
).set_index(
    df
    .index.rename_categories(['red', 'green', 'blue'])
    .rename('color'), append=True
)

df.sample(5)

sepal length (cm)  sepal width (cm)  \
target target_name color                                        
2      virginica   blue                 6.9               3.2   
                   blue                 6.4               3.2   
0      setosa      red                  5.1               3.7   
1      versicolor  green                5.0               2.3   
                   green                6.3               2.5   

                          petal length (cm)  petal width (cm)  
target target_name color                                       
2      virginica   blue                 5.7               2.3  
                   blue                 5.3               2.3  
0      setosa      red                  1.5               0.4  
1      versicolor  green                3.3               1.0  
                   green                4.9               1.5

In [4]:
test_train = next(
    sklearn.model_selection
    .StratifiedKFold(n_splits=2)
    .split(df.values, df.Index('target'))
)

split = DataFrame(index=df.index, columns=['split'])
split.iloc[test_train[0]], split.iloc[test_train[1]] = 'train', 'test'
df = df.set_index(split.set_index('split', append=True).swaplevel(-1,0).index)

df.ix['train'].fit().sample(5)

sepal length (cm)  sepal width (cm)  \
target_name color target                                        
setosa      red   0                     4.9               3.1   
virginica   blue  2                     6.8               3.2   
setosa      red   0                     4.8               3.0   
                  0                     5.2               3.4   
versicolor  green 1                     6.0               2.9   

                          petal length (cm)  petal width (cm)  
target_name color target                                       
setosa      red   0                     1.5               0.1  
virginica   blue  2                     5.9               2.3  
setosa      red   0                     1.4               0.3  
                  0                     1.4               0.2  
versicolor  green 1                     4.5               1.5

In [5]:
from sklearn.metrics import confusion_matrix

In [6]:
from sklearn import preprocessing, decomposition, discriminant_analysis

In [7]:
df = (
    df
    .set_params(estimator=preprocessing.RobustScaler())
    .fit_transform()
    .set_params(estimator=decomposition.PCA())
    .fit_transform()
    .set_params(estimator=discriminant_analysis.LinearDiscriminantAnalysis())
)
df.ix['train'].fit().sample(2)

0         1         2         3
target_name color target                                        
versicolor  green 1      -0.795941 -0.291272  0.084160  0.093812
                  1      -0.433387 -0.054525  0.058947  0.018300

In [8]:
(
    df
    .transform()
    .set_index(
        CategoricalIndex(df.predict()[0], name='prediction'), append=True
    )
    .sample(2)
)

,,,,,0,1
split,target_name,color,target,prediction,,
test,setosa,red,0,0,7.341003,-0.791289
train,versicolor,green,1,1,-2.198081,1.041751


In [9]:
df.ix['train'].score()

0.97333333333333338

In [10]:
df.add_template(
scoreboard="""
The table presents some information about the `{{
    df.estimator.__str__().split('(',1)[0]
}}` model applied to the `iris` dataset.

|         |           Test          |           Training       |
|---------|-------------------------|--------------------------|
| Score   |{{df.ix['test'].score()}}|{{df.ix['train'].score()}}|
| Samples | {{df.ix['test'] | len}} | {{df.ix['train'] | len}} |
"""
);

In [11]:
Markdown(df.get_template('scoreboard'))


The table presents some information about the `LinearDiscriminantAnalysis` model applied to the `iris` dataset.

|         |           Test          |           Training       |
|---------|-------------------------|--------------------------|
| Score   |0.986666666667|0.973333333333|
| Samples | 75 | 75 |

In [12]:
transformed = df.transform()
transformed.sample(2)

,,,,0,1
split,target_name,color,target,,
train,virginica,blue,2,-4.878197,-0.030116
test,virginica,blue,2,-4.693366,-1.228449


In [13]:
with transformed.reset_index().DataSource(
    x=0, y=1, fill_color='color', text='target_name'
) as source:
    source.Scatter(color='text', marker='split')
    source.save()